In [15]:
from jupyter_dash import JupyterDash
from dash import Dash, html,dcc,Input, Output
import dash_leaflet as dl
import json
import httpx
from rio_tiler.io import COGReader
import xarray as xr

In [16]:
titiler_endpoint = "https://mfkh268xng.execute-api.us-east-1.amazonaws.com/"  # personal endpoint. Please be kind.
url = "https://rapidepe123.s3.eu-west-2.amazonaws.com/GBR_wind-speed_100m_COGv2.tif"


In [20]:

# Fetch File Metadata to get min/max rescaling values (because the file is stored as float32)
r = httpx.get(
    f"{titiler_endpoint}/cog/info",
    params = {
        "url": url,
    }
).json()

print(r)
print("Data is of type:", r["dtype"])

# This dataset has statistics metadata
minv, maxv = r["band_metadata"][0][1]["STATISTICS_MINIMUM"], r["band_metadata"][0][1]["STATISTICS_MAXIMUM"]
print("With values from ", minv, "to ", maxv)

{'detail': 'Out of range float values are not JSON compliant'}


KeyError: 'dtype'

In [ ]:
# We could get the min/max values using the statistics endpoint
r = httpx.get(
    f"{titiler_endpoint}/cog/statistics",
    params = {
        "url": url,
    }
).json()

print(json.dumps(r["1"], indent=4))

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "url": url,
        "rescale": f"{minv},{maxv}",
        "colormap_name": "viridis"
    }
).json()


In [ ]:
def get_point_value(lat,lon):

    with COGReader(url) as cog:
        values = cog.point(lon,lat)
        
    return "{:.2f}".format(float(values[0]))
    

x = get_point_value(55.5,-4)
print(x)

10.10


In [ ]:
app = JupyterDash(__name__)


info = html.Div( id="info", className="info",
                style={"position": "absolute", "bottom": "10px", "left": "10px", "z-index": "1000"})


app.layout = html.Div([
dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[55, -4],
               zoom=5,
               id = "map",
               children=[
            dl.LayersControl([
                
                   dl.Overlay(dl.LayerGroup(dl.TileLayer(url="https://cartodb-basemaps-{s}.global.ssl.fastly.net/light_nolabels/{z}/{x}/{y}.png",id="TileMap")),name="BaseMap",checked=True),
                   dl.Overlay(dl.LayerGroup(dl.TileLayer(url=r["tiles"][0], opacity=0.8,id="WindSpeed@100m")),name="WS@100m",checked=True),
                   dl.LayerGroup(id="layer"),
                                
                   dl.Colorbar(colorscale="viridis", width=20, height=150, min=minv, max=maxv,unit='m/s',position="bottomright"),
                   info,
                   
])
])
])

@app.callback(Output("info", "children"), [Input("map", "click_lat_lng")])
def map_click(click_lat_lng):

    lat= click_lat_lng[0]
    lon=click_lat_lng[1]
    
    return get_point_value(lat,lon)



app.run_server(debug=True,mode='inline')
